##### Step 1: Convert CSV to JSON Files

In [ ]:
import csv
import json
import uuid
import os

raw_reviews_file = "../data/raw/hotel_reviews_1000.csv"
transformed_dir = "../data/transformed"

raw_reviews = open(raw_reviews_file, "r").readlines()

if not os.path.exists(transformed_dir):
    os.makedirs(transformed_dir)

def process_reviews(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        # Read the first line to get the header
        header = next(csv.reader(csvfile))
        
        # Create a mapping of expected column names to actual column names
        column_mapping = {
            'dateAdded': 'dateAdded',
            'city': 'city',
            'hotel_name': 'name',
            'hotel_state': 'province',
            'review_text': 'reviews.text',
            'review_title': 'reviews.title'
        }
        
        # Find the index of each required column
        column_indices = {}
        for expected_name, actual_name in column_mapping.items():
            try:
                column_indices[expected_name] = header.index(actual_name)
            except ValueError:
                print(f"Warning: Column '{actual_name}' not found in the CSV. Some data may be missing.")
        
        # Reset file pointer to the beginning
        csvfile.seek(0)
        
        # Skip the header row
        next(csvfile)
        
        # Use csv.reader instead of DictReader
        reader = csv.reader(csvfile)
        
        for i, row in enumerate(reader, start=1):
            review_json = {}
            for key, index in column_indices.items():
                if index < len(row):
                    review_json[key] = row[index]
                else:
                    review_json[key] = ""  # or None, depending on your preference
            
            # Generate a unique identifier
            review_json['id'] = str(uuid.uuid4())
            
            # print(json.dumps(review_json, indent=2))
            print(f"processed record [{i}] with id [{review_json['id']}]")

            with open(f"{transformed_dir}/review_{i}.json", "w+") as f:
                json.dump(review_json, f, indent=2)
            
process_reviews(raw_reviews_file)

#### Step 2: Create Embeddings for each of the JSON Files

In [2]:
%pip install -q python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

response = client.embeddings.create(
    input="Hello world",
    model="text-embedding-3-small"
)

print(len(response.data[0].embedding))
print(response.data[0].embedding)

1536
[-0.002119065960869193, -0.04909009113907814, 0.02101006731390953, 0.03132298216223717, -0.04530920088291168, -0.02640475146472454, -0.02897145412862301, 0.060248326510190964, -0.025713125243782997, -0.01480848528444767, 0.015446318313479424, -0.03007805533707142, -0.02039528824388981, -0.03341323137283325, 0.025820711627602577, 0.014255184680223465, -0.070023313164711, 0.012364739552140236, 0.014816170558333397, 0.04884418100118637, 0.020764155313372612, -0.008829761296510696, -0.015069766901433468, -0.01661439798772335, 0.025959037244319916, -0.002841430949047208, -0.024391351267695427, 0.024283764883875847, 0.0018039917340502143, -0.0557297021150589, 0.023069577291607857, -0.045462895184755325, -0.008714490570127964, 0.0031622687820345163, 0.0045263092033565044, 0.0017943857237696648, 0.026696771383285522, 0.010159220546483994, -0.011995872482657433, -0.01151941902935505, -0.014885333366692066, -0.02314642444252968, 0.025390366092324257, 0.03682525455951691, -0.0355034768581390

In [4]:
import os
import json

transformed_dir = "../data/transformed"
embedded_dir = "../data/embedded"

if not os.path.exists(embedded_dir):
    os.makedirs(embedded_dir)
    
def prepare_embedding_str(review_json):
    return f"REVIEW_TITLE: {review_json['review_title']} REVIEW_TEXT: {review_json['review_text']} HOTEL_NAME: {review_json['hotel_name']} HOTEL_CITY: {review_json['city']} HOTEL_STATE: {review_json['hotel_state']}"
    
client = OpenAI()
for file in os.listdir(transformed_dir):
    with open(f"{transformed_dir}/{file}", "r") as f:
        review = json.load(f)
        
        ## start here
        embedding_str = prepare_embedding_str(review)
        response = client.embeddings.create(
            input=embedding_str,
            model="text-embedding-3-small"
        )
        
        review['embedding'] = response.data[0].embedding
        
        with open(f"{embedded_dir}/{file}", "w") as f:
            json.dump(review, f, indent=2)